# Токенизатор

Токенизатор основан на алгоритме BPE. \
Дополнительно для более качественной токенизации применяется несколько ограничений на слияние токенов. \
\
Запрещено слияние:
- для объединения в один токен нескольких слов (при этом, токены из нескольких пробелов допускаются) 
- если один токен - буквенный, другой - нет. Несколько примеров:
  - " help" + "ful" - можно
  - " help" + "ful." - нельзя
  - ".." + "." - можно
- если один токен - цифровой, другой - нет. (запрещает токены вроде "2021.", "2021,")
- если один из токенов - символ новой строки (запрещает токены вроде "\nExternal")
  
\
Я обучил два токенизатора:
1. Учитывает регистр. 25000 слияний. Словарь: `checkpoints/tokenizer/tokenizer_25k_10k.vocab`
2. Не учитывает регистр. 15000 слияний. Словарь: `checkpoints/tokenizer/tokenizer_15k_10k_uncased.vocab`

Оба обучены на 10000 статей из вики.

In [2]:
text = """
What is a piece of text?
A text is a passage of words that conveys a set of meanings to the person who is reading it. 
It’s a body of written work, in various forms and structures, that can be words, phrases and sentences that piece together a passage of written work.
To put it as simply as possible, it is a group of words. But it can come in many different forms.
A text can be written materials, such as books, magazines, newspapers, or online content. 
But it can also be other things, those that we may not associate with standard text. 
Text could be movies, scripts, paintings, songs, political cartoons, advertisements and maps. 
If we can look at something with words and sentences, explore it, find layers of meaning in it, and draw information and conclusions from it, you’re looking at a text."""

In [3]:
from modules.tokenizer import Tokenizer

print("cased tokenization:")
cased_tokenizer = Tokenizer.init_and_load("checkpoints/tokenizer/tokenizer_25k_10k.pkl")
cased_tokenizer.visualize(text)

print("uncased tokenization:")
uncased_tokenizer = Tokenizer.init_and_load("checkpoints/tokenizer/tokenizer_15k_10k_uncased.pkl")
uncased_tokenizer.visualize(text)

cased tokenization:

What is a piece of text?
A text is a passage of words that conveys a set of meanings to the person who is reading it. 
It’s a body of written work, in various forms and structures, that can be words, phrases and sentences that piece together a passage of written work.
To put it as simply as possible, it is a group of words. But it can come in many different forms.
A text can be written materials, such as books, magazines, newspapers, or online content. 
But it can also be other things, those that we may not associate with standard text. 
Text could be movies, scripts, paintings, songs, political cartoons, advertisements and maps. 
If we can look at something with words and sentences, explore it, find layers of meaning in it, and draw information and conclusions from it, you’re looking at a text.
uncased tokenization:

what is a piece of text?
a text is a passage of words that conveys a set of meanings to the person who is reading it. 
it’s a body of written work, i

# Модель

Модель совпадает с трансформер-декодером из "Attention is all you need", за исключением того, что LayerNorm теперь находится перед слоями, а не после.

Также используется weight sharing между слоем эмбеддингов и финальной проекцией.

Используется инициализация линейных слоёв $\sim N\left(0, \frac{0.02}{\sqrt{2 \cdot depth}}\right)$, благодаря чему получаются одинаковые дисперсии активаций на разных слоях при инициализаии.

In [4]:
from modules.transformer import Transformer

transformer = Transformer()

print(f"config: {transformer.config}")
print(f"params: {sum(p.numel() for p in transformer.parameters()):,}")

config: TransformerConfig(vocab_size=15256, d_model=768, context_length=512, n_heads=12, n_layers=12, p_dropout=0.1)
params: 96,744,960


In [5]:
transformer

Transformer(
  (token_embedding_table): Embedding(15256, 768)
  (positional_encoding): PositionalEncoding()
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (attn): MultiHeadAttention(
        (heads): ModuleList(
          (0-11): 12 x MaskedSelfAttention(
            (q_proj): Linear(in_features=768, out_features=64, bias=False)
            (k_proj): Linear(in_features=768, out_features=64, bias=False)
            (v_proj): Linear(in_features=768, out_features=64, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffn): Sequential(
        (0): Linear(in_features=768, out_features=3072, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=3072, out_features=768, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (ln1): LayerNorm((768,), eps=1e-05, elementwise_affine=Tr

# Данные

Данные взяты из англоязычного дампа википедии. 

Предобрабатываются скриптом `scripts/preprocess_data.py`, где разделяются на train / validation, токенизируются, паддятся, и сохраняются чанками по 10000 айтемов.

В интерфейсе доступа к данным реализовано кэширование этих чанков, что при последовательном доступе к данным минимизирует расходы на i/o.

In [8]:
from modules.data import WikipediaTokenizedDataset 

dataset = WikipediaTokenizedDataset("data/uncased-15k-10k/test")
dataset[0].keys()

dict_keys(['x', 'y', 'pad_mask'])

# Обучение

При обучении использовались следующий гиперпарметры:
- lr = 6e-4
- batch = 8
- grad_accum = 2 -> 32 (увеличивался по мере обучения)
- weight_decay = 0.1 для линейных слоёв

Таким образом, количество токенов на одну оптимизацию было 8192 в начале и 131072 в конце

Также использовался косинусный scheduler с линейным warmup на первые 10% шагов 

Всего было проведено 5000 шагов оптимизации (что, конечно, очень мало)

Скрипт обучения: `scripts/train_transformer.py`

Логи обучения: `train_log.txt`

# Генерация

In [6]:
transformer = Transformer.init_and_load("checkpoints/transformer_uncased_5000_steps/ckpts/model_5000.pt")

prompts = [
    "a text is a passage of words that conveys a set of meanings.",
    "A text can be written materials",
]

inputs, pad_mask = uncased_tokenizer(prompts)
inputs, pad_mask

(tensor([[   97,  3055,   336,   257,  9857,   283,  5101,   384, 12112,   115,
            257,   953,   283,  2679,   657,    46],
         [   97,  3055,   614,   361,  1913,  6061,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0]]),
 tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True, False, False, False, False,
          False, False, False, False, False, False]]))

In [9]:
# top_k = 50 by default
generated_tokens = transformer.generate(inputs, 10, attn_mask=pad_mask)
uncased_tokenizer.decode_batch(generated_tokens)

['a text is a passage of words that conveys a set of meanings. that in 2010, a  references, ',
 'a text can be written materials, to and to the former and he as an']

## greedy

In [9]:
generated_tokens = transformer.generate(inputs, 10, attn_mask=pad_mask, greedy=True)
uncased_tokenizer.decode_batch(generated_tokens)

['a text is a passage of words that conveys a set of meanings. =========',
 'a text can be written materials, the the the the the the the the the']

## top-k

In [10]:
generated_tokens = transformer.generate(inputs, 10, attn_mask=pad_mask, top_k=50)
uncased_tokenizer.decode_batch(generated_tokens)

['a text is a passage of words that conveys a set of meanings. she. the the same she when ==',
 'a text can be written materials-american a also he of the been the the']

## top-p

In [12]:
generated_tokens = transformer.generate(inputs, 10, attn_mask=pad_mask, top_p=0.9, top_k=None)
uncased_tokenizer.decode_batch(generated_tokens)

["a text is a passage of words that conveys a set of meanings. or well in congo's =13 ",
 'a text can be written materials single their average was australian the she eastern toronto too']

## temperature

In [13]:
# low
generated_tokens = transformer.generate(inputs, 10, attn_mask=pad_mask, temperature=0.5)
uncased_tokenizer.decode_batch(generated_tokens)

['a text is a passage of words that conveys a set of meanings. in the the the the the the the the the',
 'a text can be written materials. in the a the an the the a the']

In [14]:
# high
generated_tokens = transformer.generate(inputs, 10, attn_mask=pad_mask, temperature=2.5)
uncased_tokenizer.decode_batch(generated_tokens)

['a text is a passage of words that conveys a set of meanings. as most would time that a not the this were',
 'a text can be written materials. was it the this by an 12 at']

## repetition penalty

In [15]:
# low
generated_tokens = transformer.generate(inputs, 10, attn_mask=pad_mask, repetition_penalty=0.1)
uncased_tokenizer.decode_batch(generated_tokens)

['a text is a passage of words that conveys a set of meanings. a a a a a a a a a a',
 'a text can be written materialsaaaaaaaaaa']

In [16]:
# high
generated_tokens = transformer.generate(inputs, 10, attn_mask=pad_mask, repetition_penalty=2.0)
uncased_tokenizer.decode_batch(generated_tokens)

['a text is a passage of words that conveys a set of meanings. the ( was born he from =–',
 "a text can be written materials's a known. he who as film of"]